In [33]:
import pandas as pd

COLUMNS_TO_TAKE = {"var": "variable", "var_desc": "variable_description", "category": "category"}

variable_description_raw = pd.read_csv("../fusion/data/VarDescription.csv", usecols=COLUMNS_TO_TAKE).rename(columns=COLUMNS_TO_TAKE)
variable_description = variable_description_raw.drop_duplicates().set_index("variable")

In [70]:
import os 
from tqdm import tqdm

for main_category in tqdm(["examination", "laboratory", "demographics"]):
    missing_entire_files = {}
    list_missing_variables_description = []
    for idx, file in enumerate(os.listdir(f"../extraction/data/{main_category}/")):
        if file[-4:] == ".csv":
            variables = pd.read_csv(f"../extraction/data/{main_category}/{file}").columns.drop(["SEQN", "file_name", "cycle", "begin_year", "end_year"])

            missing_variables = variables[~variables.isin(variable_description.index)]

            if len(missing_variables) == len(variables):
                missing_entire_files[file] = variables.tolist()
            elif len(missing_variables) > 0:
                list_missing_variables_description.append(dict(zip(missing_variables.tolist(), [file] * len(missing_variables))))
        
        if idx > 10:
            break
    break

missing_variables_description = {k: v for missing_variables_description in list_missing_variables_description for k, v in missing_variables_description.items()}

  0%|          | 0/3 [00:00<?, ?it/s]/n/groups/patel/theo/NHANES_preprocessing/env_o2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (68,72,73,77,78,82,83,84,85,86,87,92,95,97,100,102,103,104,105,108,109,110,111,112,113,115,117) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
  0%|          | 0/3 [00:02<?, ?it/s]


In [43]:

import json
with open(f"../fusion/splitting/missing_classification.json", "w") as outfile:
    json.dump(missing_description.loc[missing_list].to_dict("index"), outfile)